<a href="https://colab.research.google.com/github/ddekun/NLP/blob/lesson7/lesson7/hw7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Введение в обработку естественного языка

### Урок 7. Сверточные нейронные сети для анализа текста

Задание Берем отзывы за лето (из архива с материалами или предыдущего занятия)

Учим conv сеть для классификации
Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [1]:
!pip install stop_words
!pip install pymorphy2
!pip install navec

In [2]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
# pd.set_option('display.float_format', '{:.2f}'.format)

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re
from spacy.lang.ru.stop_words import STOP_WORDS
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, get_file
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.losses import categorical_crossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Constant

In [3]:
!pip install xlrd==2.0.1

In [4]:
max_words = 2000
max_len = 100

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data = pd.read_excel('/content/drive/MyDrive/Geekbrains/Introduction to natural language processing/Lesson7/отзывы за лето.xls')
le = LabelEncoder()
data['Rating'] = le.fit_transform(data['Rating'])
data.sample(5)

,Rating,Content,Date
16504,4,Очень удобно,2017-07-17
6333,4,"Приложение удобное, работает быстро",2017-08-04
2013,3,Почему-то после ввода 5значного кода просит вв...,2017-08-11
16381,3,Иногда приходится вводить пароль по два раза. ...,2017-07-18
15296,4,"Все супер!даже не думала,что так легко,не выхо...",2017-07-22


In [6]:
sw = set((get_stop_words("ru") + list(STOP_WORDS))) - {'не', 'ни', 'нет'}
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt, exclude=exclude, sw=sw, morpher=morpher):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [7]:
data['preprocessed_text'] = data['Content'].apply(preprocess_text)
data.sample(5)

,Rating,Content,Date,preprocessed_text
12661,4,Все отлично,2017-07-26,отлично
20611,3,После обновления очень долго стало загружаться,2017-06-02,обновление загружаться
14157,4,Уберите строчку с кредитами - у мегя их нет,2017-07-24,убрать строчка кредит мегть нет
12387,4,Очень удобно. Не надо не куда ходить.,2017-07-27,удобно не не ходить
15512,4,Очень хорошо,2017-07-21,


In [8]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
train_corpus = " ".join(data["preprocessed_text"])
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]
len(tokens_filtered)

102261

In [10]:
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'не',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'телефон',
 'отличный']

In [11]:
vocabulary = {v: k for k, v in enumerate(tokens_filtered_top, 1)}

In [12]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [13]:
data_train = np.asarray([text_to_sequence(text, max_len) for text in data['preprocessed_text']], dtype=np.int32)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data_train, data['Rating'], test_size=0.3)

In [15]:
batch_size = 1024
epochs = 10

In [16]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

In [17]:
optimizer = Adam(learning_rate=0.001)
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer=optimizer,
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          256000    
                                                                 
 conv1d (Conv1D)             (None, 96, 128)           82048     
                                                                 
 activation (Activation)     (None, 96, 128)           0         
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 20)                2580      
                                                                 
 activation_1 (Activation)   (None, 20)                0         
                                                        

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=0,
                    validation_split=0.1)

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
!pip install natasha
import natasha

In [ ]:
from natasha import (
Segmenter,
MorphVocab,
NewsEmbedding,
NewsMorphTagger,
NewsSyntaxParser,
NewsNERTagger,
PER,
NamesExtractor,
Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

In [ ]:
embedding_dim = 300

embedding_matrix = np.zeros((max_words, embedding_dim))
for word in tokens_filtered_top:
    embedding_vector = emb.get(word)
    if embedding_vector is not None:
        embedding_matrix[tokens_filtered_top.index(word)] = embedding_vector

In [ ]:
embedding_matrix.shape

In [ ]:
embedding_layer = Embedding(
    max_words,
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    trainable=False,
)

In [ ]:
model = Sequential()
model.add(embedding_layer)
model.add(Conv1D(128, 5))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(20))
model.add(Activation("relu"))
model.add(Dropout(0.25))
model.add(Dense(5))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=0,
          validation_split=0.1)

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])